In [17]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os.path
import mediapipe as mp
import time
import os
from support.funcs import *

try:
    os.remove('posout.npy')
except:
    pass
h=720 
w=1280
fps=30
windowscale=0.6

LH, LK, LA, LT, RH, RK, RA, RT = [],[],[],[],[],[],[],[]

mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
 
# Initializing the drawing utils for drawing the facial landmarks on image
mp_drawing = mp.solutions.drawing_utils

pc = rs.pointcloud()


try:
    # Create pipeline
    pipeline = rs.pipeline()

    # Create a config object
    config = rs.config()

    # Tell config that we will use a recorded device from file to be used by the pipeline through playback.
    rs.config.enable_device_from_file(config, r'C:\Users\arpan\OneDrive\Documents\internship\bags\test.bag', repeat_playback=False)

    # Configure the pipeline to stream the depth stream
    # Change this parameters according to the recorded bag file resolution
    config.enable_stream(rs.stream.depth, rs.format.z16, fps)
    config.enable_stream(rs.stream.color, rs.format.rgb8, fps)

    # Start streaming from file
    profile=pipeline.start(config)
    
    # Create colorizer object
    colorizer = rs.colorizer()

    #Needed so frames don't get dropped during processing:
    profile.get_device().as_playback().set_real_time(False)

    align_to = rs.stream.color
    align = rs.align(align_to)

    timestamps=[]
        
    c=0
    # Streaming loop
    while True:

        frame_present, frameset = pipeline.try_wait_for_frames()
    
        #End loop once video finishes
        if not frame_present:
            break

        # Wait for a coherent pair of frames: depth and color
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        # get timestamp of frame
        timestamps.append((rs.frame.get_frame_metadata(depth_frame,rs.frame_metadata_value.time_of_arrival))/1000)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame() 
        color_frame = aligned_frames.get_color_frame()

        # Convert images to numpy arrays
        depth_image = np.asanyarray(aligned_depth_frame.get_data()) 
        color_image = np.asanyarray(color_frame.get_data()) 

        # Making predictions using holistic model
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        color_image.flags.writeable = False
        results = holistic_model.process(color_image)
        color_image.flags.writeable = True

        #converting color image to BGR
        color_image = cv2.cvtColor(color_image, cv2.COLOR_RGB2BGR)

        # Drawing the pose landmarks
        mp_drawing.draw_landmarks(
        color_image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS)

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)

        depth_colormap_dim = depth_colormap.shape
        color_colormap_dim = color_image.shape
        

        dic = {}
        for mark, data_point in zip(mp_holistic.PoseLandmark, results.pose_landmarks.landmark):
            dic[mark.value] = dict(landmark = mark.name, 
                x = data_point.x,
                y = data_point.y)        
        try:
            LH.append([dic[23]['x']*w,dic[23]['y']*h])
        except:
            LH.append(np.nan)
        try:
            LK.append([dic[25]['x']*w,dic[25]['y']*h])
        except:
            LK.append(np.nan)
        try:
            LA.append([dic[27]['x']*w,dic[27]['y']*h])
        except:
            LA.append(np.nan)
        try:
            LT.append([dic[31]['x']*w,dic[31]['y']*h])
        except:
            LT.append(np.nan)
        try:
            RH.append([dic[24]['x']*w,dic[24]['y']*h])
        except:
            RH.append(np.nan)
        try:
            RK.append([dic[26]['x']*w,dic[26]['y']*h])
        except:
            RK.append(np.nan)
        try:
            RA.append([dic[28]['x']*w,dic[28]['y']*h])
        except:
            RA.append(np.nan)
        try:
            RT.append([dic[32]['x']*w,dic[32]['y']*h])
        except:
            RT.append(np.nan)
        try:
            draw_box(color_image,LH[c],LK[c])
            draw_box(color_image,LK[c],LA[c])
            draw_box(color_image,LA[c],LT[c])
            
            draw_box(color_image,RH[c],RK[c])
            draw_box(color_image,RK[c],RA[c])
            draw_box(color_image,RA[c],RT[c])
        except:
            pass


        cv2.putText(color_image, str(int(c)), (50, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        color_image=cv2.resize(color_image, (int(w*windowscale),int(h*windowscale)))
        depth_colormap=cv2.resize(depth_colormap, (int(w*windowscale),int(h*windowscale)))

        # If depth and color resolutions are different, resize color image to match depth image for display
        if depth_colormap_dim != color_colormap_dim:
            resized_color_image = cv2.resize(color_image, dsize=(depth_colormap_dim[1], depth_colormap_dim[0]), interpolation=cv2.INTER_AREA)
            images = np.hstack((resized_color_image, depth_colormap))
        else:
            images =np.hstack((color_image, depth_colormap))
        
        # Show images
        cv2.putText(color_image, str(int(c)), (50, 70), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

        mapped_frame = color_frame
        pc.map_to(mapped_frame)
        
        try:
            points = pc.calculate(aligned_depth_frame)
            v = points.get_vertices()
            verts = np.asanyarray(v).view(np.float32)
            xyzpos=verts.reshape(h,w, 3)  # xyz
            xyzpos=xyzpos.astype(np.float16)
            with open('posout.npy', 'ab') as f:
                np.save(f,xyzpos)   
                # print(c)         
        except:
            print(type(v))
        
        c+=1

finally:

    # Stop streaming
    pipeline.stop()
    cv2.destroyAllWindows()
    print(c)

161


RuntimeError: Frame didn't arrive within 5000

In [ ]:
import pandas as pd
import numpy as np

land_marks={'LH':LH,'LK':LK,'LA':LA,'LT':LT,'RH':RH,'RK':RK,'RA':RA,'RT':RT}

pos=[]
with open('posout.npy', 'rb') as f:
    for i in range(c):
        y=np.load(f)
        pos.append(y)

pos=np.array(pos)
print(pos.shape)
df=pd.DataFrame()
xyz=['_x','_y','_z']

df['epoch_time']=pd.Series(timestamps)

for key,value in land_marks.items():    
    for j in range(3):
        data=[]
        for i in range(len(pos)):
            try:
                x=pos[i][int((value[i][1]))][int((value[i][0]))][j]
                data.append(x)
            except:
                continue
        df[key+xyz[j]]=pd.Series(data)
    
df

(162, 720, 1280, 3)


,epoch_time,LH_x,LH_y,LH_z,LK_x,LK_y,LK_z,LA_x,LA_y,LA_z,...,RH_z,RK_x,RK_y,RK_z,RA_x,RA_y,RA_z,RT_x,RT_y,RT_z
0,1.686721e+09,-0.138428,-0.031281,2.398438,-0.073975,0.367676,2.755859,-0.013580,0.732422,2.816406,...,2.486328,-0.207031,0.477295,2.562500,-0.306396,0.880371,2.568359,-0.275146,0.938477,2.439453
1,1.686721e+09,-0.100281,-0.027618,2.312500,-0.001148,0.369873,2.748047,-0.013542,0.730469,2.808594,...,2.371094,-0.247559,0.458008,2.486328,-0.304688,0.867188,2.529297,-0.267090,0.933105,2.439453
2,1.686721e+09,-0.097046,-0.018250,2.421875,-0.001119,0.398438,2.675781,-0.013771,0.749023,2.855469,...,2.246094,-0.274902,0.463379,2.486328,-0.307617,0.861328,2.529297,-0.285889,0.937500,2.486328
3,1.686721e+09,-0.096558,-0.010193,2.410156,-0.010147,0.411865,2.726562,-0.013618,0.753418,2.824219,...,2.361328,-0.290527,0.462158,2.480469,-0.303955,0.862305,2.523438,-0.274170,0.937988,2.480469
4,1.686721e+09,-0.095886,-0.002211,2.394531,-0.016022,0.402588,2.703125,-0.013580,0.748047,2.816406,...,2.382812,-0.297852,0.454102,2.451172,-0.308350,0.849609,2.535156,-0.276367,0.917969,2.427734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.686721e+09,-0.096130,-0.006931,2.216797,-0.079346,0.447266,2.371094,-0.095093,0.866211,2.511719,...,2.232422,-0.311279,0.450195,2.388672,-0.295654,0.878906,2.523438,NaN,NaN,NaN
158,1.686721e+09,-0.095337,-0.004566,2.255859,0.020813,0.626465,3.361328,-0.001119,0.911133,2.675781,...,2.277344,-0.277832,0.457764,2.371094,-0.280273,0.863281,2.486328,NaN,NaN,NaN
159,1.686721e+09,-0.082031,-0.002060,2.232422,0.045288,0.434814,2.333984,0.043518,0.809570,2.376953,...,2.193359,-0.255615,0.458984,2.312500,-0.283936,0.885742,2.542969,NaN,NaN,NaN
160,1.686721e+09,0.010727,0.025909,2.107422,0.129639,0.446045,2.421875,0.132690,0.781250,2.427734,...,2.207031,-0.197144,0.471680,2.312500,-0.283203,0.882812,2.511719,NaN,NaN,NaN


In [ ]:
startflag=0
before_occ=0
do=False
box_dic={'lul': [LH,LK,['LH','LK']],'rul': [RH,RK,['RH','RK']],
         'lll': [LK,LA,['LK','LA']],'rll': [RK,RA,['RK','RA']],
         'lfb': [LA,LT,['LA','LT']],'rfb': [RA,RT,['RA','RT']]}

for k,j in land_marks.items():
    for key,values in box_dic.items():
        for i in range(c):
            r= 40 if key == 'lhb' or key == 'rhb' else 30
            try:
                if point_in_quad(j[i],draw_box(color_image,values[0][i],values[1][i],(0,0,1),r)) and k not in values[2]:
                    if startflag == 0:
                        startflag = i
                        before_occ=startflag-1 # before occlusion
                    for p in values[2]:
                        if df[k+'_z'].tolist()[before_occ]>df[p+'_z'].tolist()[before_occ]:
                            print(k,'is occluded by', key, p , 'at frame', i) 
                            # df.loc[i,k+'_x']=df.loc[before_occ,k+'_y']
                            # df.loc[i,k+'_y']=df.loc[before_occ,k+'_x']
                            df.loc[i,k+'_z']=df.loc[before_occ,k+'_z']
                    do=True
            except:
                pass
            else:
                do=False
        if do:
            startflag=0

LH is occluded by rul RH at frame 0
RH is occluded by lul LH at frame 0


In [ ]:
df.to_csv('mpipe.csv', index=False)
print(df.columns)


Index(['epoch_time', 'LH_x', 'LH_y', 'LH_z', 'LK_x', 'LK_y', 'LK_z', 'LA_x',
       'LA_y', 'LA_z', 'LT_x', 'LT_y', 'LT_z', 'RH_x', 'RH_y', 'RH_z', 'RK_x',
       'RK_y', 'RK_z', 'RA_x', 'RA_y', 'RA_z', 'RT_x', 'RT_y', 'RT_z'],
      dtype='object')
